In [1]:
import re
import glob
import json
import string
import gensim.models

In [2]:
def ouvrir_json(chemin):
    f = open(chemin, encoding="utf−8") 
    toto = json.load(f)
    f.close()
    return toto
def ecrire_json(dataset,chemin):
    donnee = json.dumps(dataset, indent =2,ensure_ascii=False) 
    w = open(chemin, "w",encoding = "utf-8")
    w.write(donnee)
    w.close()
    print("Jeu de données stocké dans %s"%chemin)

In [3]:
def preprocess(s):
	s_without_punct = s
	for char in string.punctuation:
		s_without_punct = s_without_punct.replace(char, ' ') # on replace les ponctuations par des espaces
	tokens = s_without_punct.split()
	return tokens

In [4]:
def get_corpus(corpus_path):
	# assume there's one sentence per line, tokens separated by whitespace
	# simple_preprocess: "Convert a document into a list of lowercase tokens, 
	# ignoring tokens that are too short or too long.""
	with open(corpus_path, mode='r', encoding='utf-8') as inFile:
		return [preprocess(line) for line in inFile.readlines()]

In [5]:
class Embeddings(object):
	"""Programs to learn, save and load embeddings."""
	def __init__(self, corpus_path, model_path):
		self._corpus_path = corpus_path
		self._model_path = model_path

	def learn(self):
		sentences = get_corpus(self._corpus_path) # List of sentences (a sentence = list of words)
		self._model = gensim.models.Word2Vec(sentences=sentences)
		self._model.save(self._model_path)

	def learn_restrictive(self, min_freq):
		sentences = get_corpus(self._corpus_path) # List of sentences (a sentence = list of words)
		self._model = gensim.models.Word2Vec(sentences=sentences, size=100,window= 20, min_count=min_freq, workers=4)
		self._model.save(self._model_path)

	def load(self):
		self._model = gensim.models.Word2Vec.load(self._model_path)

	def most_similar(self, word, topN=10):
		return self._model.wv.most_similar(positive=[word], topn=topN)

	def most_similar_analogy(self, pos, neg, topN=10):
		return self._model.wv.most_similar(positive=pos, negative=neg, topn=topN)

	def similarity(self, w1, w2):
		if w1 in self._model.wv and w2 in self._model.wv:
			return self._model.wv.similarity(w1, w2)
		else:
			return -1

	def get_vocab(self):
		return self._model.wv.vocab

	def get_vector(self, word):
		return self._model.wv.get_vector(word)

	def get_model(self):
		return self._model
		

In [37]:
#corpus_n : corpus "normal"(du type faq)
#corpus_s corpus "scientifique"

In [8]:
f1=open("output/corpus_n.txt","w",encoding="utf8")
for chemin in  glob.glob('corpus/appariements_faq\*.*')  :
    #print(chemin)
    corpus_q_r = ouvrir_json(chemin)
    for k,v in corpus_q_r.items():
        for l in v :
            if l != " ":
                f1.write(l)
f1.close( )

In [9]:
f2=open("output/corpus_s.txt","w",encoding="utf8")

for chemin in  glob.glob('corpus/corpus_TALN_2007-2013\*')  : #il faudra d'aboard décompresser le dossier "corpus_TALN_2007-2013"
    f = open(chemin,"r",encoding="utf-8")
    corpus_s = f.read()
    f2.write(corpus_s)
f.close( )
f2.close()

In [11]:
subCorpus_path_n = "output/corpus_n.txt"
model_path_n = "output/corpus_n.W2Vmodel"
embeddings = Embeddings(subCorpus_path_n, model_path_n)
embeddings.learn ()

In [12]:
subCorpus_path_s = "output/corpus_s.txt"
model_path_s = "output/corpus_s.W2Vmodel"
embeddings = Embeddings(subCorpus_path_s, model_path_s)
embeddings.learn ()

In [13]:
embeddings_n = Embeddings(subCorpus_path_n, model_path_n)
embeddings_n.load( )
embeddings_s = Embeddings(subCorpus_path_s, model_path_s)
embeddings_s.load( )

In [14]:
import numpy as np
def cosine_similarity(vec1,vec2):
    dot = np.dot(vec1 , vec2)
    norm1 = np.linalg.norm(vec1) 
    norm2 = np.linalg.norm(vec2) 
    cos = dot / (norm1 * norm2)
    return cos # f l o a t

In [19]:
from nltk.corpus import stopwords

In [26]:
vocab_n, vocab_s = embeddings_n.get_vocab(), embeddings_s.get_vocab()
sims = {}
for word in vocab_n:
    if word in vocab_s:
        if word.lower( ) not in stopwords.words("french"):
                vec1 = embeddings_n.get_vector(word)
                vec2 = embeddings_s.get_vector(word)
                sims[word] = cosine_similarity(vec1,vec2)

In [27]:
NB_TO_SHOW = 30
sims_up = {k:v for k,v in sorted(sims.items (), key=lambda i:i[1], reverse=True )}
sims_down = {k:v for k,v in sorted(sims.items (), key=lambda i:i[1])}
print("∗∗∗ Vecteurs proches : ")
i = 0
while i < NB_TO_SHOW :
    w = list(sims_up.keys ())[i]
    print(w,end=", ")
    i += 1

print("∗∗∗ Vecteurs éloigné :")
i = 0
while i < NB_TO_SHOW :
          w = list(sims_down .keys ())[i]
          print(w, end=", ")
          i += 1


∗∗∗ Vecteurs proches : 
régulièrement, photos, publicité, accepter, frais, connaître, paiement, pouvez, dossier, merci, mail, rendez, paris, Message, XXX, visibilité, photo, contact, télécharger, inscription, droits, encore, pièces, tarifs, conseiller, comptable, Non, Merci, envoyé, administrative, ∗∗∗ Vecteurs éloigné :
via, puis, selon, alors, ainsi, avant, Si, fait, lors, partir, A, ligne, deux, cours, nouveau, statut, nouvelle, part, première, cas, propositions, proposition, fois, si, où, 1, «, peut, ci, tous, 

In [29]:
#print(sims_down)
 #conseiller comtaple droits paiement dossier etc.

In [111]:
#print(len(vocab_n))

281


In [35]:
word = "droits" #mots liés à ce "terme"
corpus = "scientifique"
sims = " ".join([w for w,s in embeddings_s.most_similar(word,topN=10)])
print("{}\t{}\t{}".format(word,corpus,sims))

droits	scientifique	métiers psychologique Guinaudeau massive témoignages dimensionnalité bruits périphériques Station physiologiques


In [36]:
word = "droits" #mots liés à ce "terme"
corpus = "normal"
sims = " ".join([w for w,s in embeddings_n.most_similar(word,topN=10)])
print("{}\t{}\t{}".format(word,corpus,sims))

droits	normal	votre les mais et dossier première qui vos scolarité du
